# Analysis of Fidelity Crossing Points as a Function of K

This notebook analyzes how the σ value at which fidelity crosses a threshold changes as a function of K (number of minimum weight matchings) for different distance values.

In [2]:
using JLD2
using Plots
# using Interpolations
using Statistics

## Load and Examine Data

In [3]:
# Load the data
data_file = "data/surface_square_mwms_5_15_0.596_0.607_400_3_1023680.jld2"
data = load(data_file)

# Display the keys to understand the structure
println("Keys in the data file:")
for key in keys(data)
    println("  $key: ", typeof(data[key]))
end

Keys in the data file:
  num_samples: Int64
  σrange: StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}
  p_list: Dict{Any, Any}
  t_list: Dict{Any, Any}
  drange: StepRange{Int64, Int64}


In [4]:
# Extract the main data dictionary (assuming it's the largest object)
data_dict = nothing
for (key, value) in data
    if isa(value, Dict) && length(value) > 10  # Heuristic to find the main data
        data_dict = value
        println("Found main data dictionary with key: $key")
        break
    end
end

if data_dict === nothing
    # If heuristic fails, let's examine all dictionaries
    println("Examining all dictionary objects:")
    for (key, value) in data
        if isa(value, Dict)
            println("  $key: Dict with $(length(value)) entries")
            if length(value) > 0
                first_key = first(keys(value))
                println("    Sample key: $first_key, value type: $(typeof(value[first_key]))")
            end
        end
    end
end

Found main data dictionary with key: p_list


In [5]:
# If the above doesn't work, let's manually select the data
# Uncomment and modify the line below if needed:
# data_dict = data["your_key_here"]

println("Data loaded successfully!")
println("Number of [σ, d] combinations: ", length(data_dict))

# Extract all unique σ and d values
all_keys = collect(keys(data_dict))
sigma_values = unique([key[1] for key in all_keys])
d_values = unique([key[2] for key in all_keys])

sort!(sigma_values)
sort!(d_values)

println("σ range: ", minimum(sigma_values), " to ", maximum(sigma_values))
println("d values: ", d_values)

# Determine K from the length of the first data vector
first_key = first(all_keys)
K_max = length(data_dict[first_key])
println("Maximum K: ", K_max)
println("Sample data point: ", data_dict[first_key][1:min(5, K_max)])

Data loaded successfully!
Number of [σ, d] combinations: 72
σ range: 0.596 to 0.607
d values: [5.0, 7.0, 9.0, 11.0, 13.0, 15.0]
Maximum K: 401
Sample data point: Any[0.841741559862457, 0.8419994529540481, 0.8440196155048453, 0.8445686151922476, 0.8450834245076586]


## Define Functions for Finding Crossing Points

In [6]:
"""
    find_crossing_sigma(sigma_values, fidelity_values, threshold=0.5)

Find the σ value where fidelity crosses a given threshold using linear interpolation.

# Arguments
- `sigma_values`: Array of σ values
- `fidelity_values`: Array of corresponding fidelity values
- `threshold`: Crossing threshold (default: 0.5)

# Returns
- The σ value at crossing, or NaN if no crossing is found
"""
function find_crossing_sigma(sigma_values, fidelity_values, threshold=0.5)
    # Remove any NaN or missing values
    valid_indices = .!isnan.(fidelity_values) .& .!ismissing.(fidelity_values)
    sigma_clean = sigma_values[valid_indices]
    fidelity_clean = fidelity_values[valid_indices]
    
    if length(sigma_clean) < 2
        return NaN
    end
    
    # Sort by sigma values
    sort_indices = sortperm(sigma_clean)
    sigma_sorted = sigma_clean[sort_indices]
    fidelity_sorted = fidelity_clean[sort_indices]
    
    # Find crossing points
    for i in 1:(length(fidelity_sorted)-1)
        f1, f2 = fidelity_sorted[i], fidelity_sorted[i+1]
        s1, s2 = sigma_sorted[i], sigma_sorted[i+1]
        
        # Check if threshold is crossed between these two points
        if (f1 <= threshold <= f2) || (f2 <= threshold <= f1)
            # Linear interpolation to find exact crossing point
            if f2 != f1  # Avoid division by zero
                crossing_sigma = s1 + (threshold - f1) * (s2 - s1) / (f2 - f1)
                return crossing_sigma
            end
        end
    end
    
    return NaN  # No crossing found
end

find_crossing_sigma

## Analyze Crossing Points for d=13 and d=15

In [7]:
function analyze_crossings_for_distance(data_dict, d, threshold=0.5)
    """
    Analyze crossing points for a specific distance d.
    """
    # Extract all keys for this distance
    all_keys = collect(keys(data_dict))
    relevant_keys = [key for key in all_keys if key[2] == d]
    
    if isempty(relevant_keys)
        println("Warning: No data found for d=$d")
        return Dict()
    end
    
    sort!(relevant_keys, by=x->x[1])  # Sort by σ
    sigmas_for_d = [key[1] for key in relevant_keys]
    
    # Determine K_max from first data point
    K_max = length(data_dict[relevant_keys[1]])
    
    crossing_sigmas = Float64[]
    k_values = Int[]
    
    println("\nAnalyzing d=$d ($(length(relevant_keys)) σ values, K_max=$K_max)...")
    
    for k in 1:K_max
        # Extract fidelity values for this K across all σ values
        fidelities = [data_dict[key][k] for key in relevant_keys]
        
        # Find crossing point
        crossing_sigma = find_crossing_sigma(sigmas_for_d, fidelities, threshold)
        
        if !isnan(crossing_sigma)
            push!(crossing_sigmas, crossing_sigma)
            push!(k_values, k)
            println("  K=$k: crossing at σ=$(round(crossing_sigma, digits=4))")
        else
            println("  K=$k: no crossing found")
        end
    end
    
    return Dict(
        "k_values" => k_values,
        "crossing_sigmas" => crossing_sigmas,
        "sigma_range" => sigmas_for_d,
        "d" => d
    )
end

# Analyze for both distances
target_distances = [13, 15]
threshold = 0.5

results = Dict()
for d in target_distances
    results[d] = analyze_crossings_for_distance(data_dict, d, threshold)
end


Analyzing d=13 (12 σ values, K_max=401)...
  K=1: no crossing found
  K=2: no crossing found
  K=3: no crossing found
  K=4: no crossing found
  K=5: no crossing found
  K=6: no crossing found
  K=7: no crossing found
  K=8: no crossing found
  K=9: no crossing found
  K=10: no crossing found
  K=11: no crossing found
  K=12: no crossing found
  K=13: no crossing found
  K=14: no crossing found
  K=15: no crossing found
  K=16: no crossing found
  K=17: no crossing found
  K=18: no crossing found
  K=19: no crossing found
  K=20: no crossing found
  K=21: no crossing found
  K=22: no crossing found
  K=23: no crossing found
  K=24: no crossing found
  K=25: no crossing found
  K=26: no crossing found
  K=27: no crossing found
  K=28: no crossing found
  K=29: no crossing found
  K=30: no crossing found
  K=31: no crossing found
  K=32: no crossing found
  K=33: no crossing found
  K=34: no crossing found
  K=35: no crossing found
  K=36: no crossing found
  K=37: no crossing found
  K

## Visualize Results

In [ ]:
# Plot crossing σ values as a function of K
p = plot(xlabel="K (number of minimum weight matchings)", 
         ylabel="Crossing σ value", 
         title="Fidelity Crossing Points vs K (threshold = $threshold)",
         legend=:topright,
         size=(800, 600))

colors = [:blue, :red, :green, :orange, :purple]
markers = [:circle, :square, :diamond, :triangle, :star]

for (i, d) in enumerate(target_distances)
    if haskey(results, d) && !isempty(results[d]["k_values"])
        plot!(p, results[d]["k_values"], results[d]["crossing_sigmas"], 
              marker=markers[i], linewidth=2, markersize=6,
              color=colors[i], label="d=$d")
    end
end

display(p)

## Detailed Analysis and Summary

In [ ]:
println("=== DETAILED SUMMARY ===")
println("Threshold used: $threshold")
println()

for d in target_distances
    if haskey(results, d) && !isempty(results[d]["k_values"])
        println("Distance d=$d:")
        k_vals = results[d]["k_values"]
        crossing_vals = results[d]["crossing_sigmas"]
        
        println("  Number of K values with crossings: $(length(k_vals))")
        println("  K range: $(minimum(k_vals)) to $(maximum(k_vals))")
        println("  σ_crossing range: $(round(minimum(crossing_vals), digits=4)) to $(round(maximum(crossing_vals), digits=4))")
        
        if length(crossing_vals) > 1
            # Calculate trend
            trend = crossing_vals[end] - crossing_vals[1]
            trend_per_k = trend / (k_vals[end] - k_vals[1])
            println("  Overall trend: σ_crossing changes by $(round(trend, digits=4)) from K=$(k_vals[1]) to K=$(k_vals[end])")
            println("  Average change per K: $(round(trend_per_k, digits=6))")
            
            # Calculate correlation
            if length(k_vals) > 2
                correlation = cor(k_vals, crossing_vals)
                println("  Correlation between K and σ_crossing: $(round(correlation, digits=4))")
            end
        end
        
        println("  Detailed values:")
        for (k, sigma) in zip(k_vals, crossing_vals)
            println("    K=$k: σ_crossing = $(round(sigma, digits=6))")
        end
        println()
    else
        println("Distance d=$d: No crossings found")
        println()
    end
end

## Optional: Visualize Fidelity Curves for Selected K Values

In [ ]:
# Plot fidelity curves for a few K values to visualize the crossing behavior
function plot_fidelity_curves(data_dict, d, k_values_to_plot=[1, 5, 10], threshold=0.5)
    all_keys = collect(keys(data_dict))
    relevant_keys = [key for key in all_keys if key[2] == d]
    sort!(relevant_keys, by=x->x[1])
    
    sigmas = [key[1] for key in relevant_keys]
    
    p = plot(xlabel="σ", ylabel="Fidelity", 
             title="Fidelity vs σ for d=$d (selected K values)",
             legend=:topright, size=(800, 600))
    
    # Add threshold line
    hline!([threshold], color=:black, linestyle=:dash, linewidth=2, label="Threshold = $threshold")
    
    colors = [:blue, :red, :green, :orange, :purple, :brown, :pink]
    
    for (i, k) in enumerate(k_values_to_plot)
        if k <= length(data_dict[relevant_keys[1]])
            fidelities = [data_dict[key][k] for key in relevant_keys]
            plot!(p, sigmas, fidelities, 
                  marker=:circle, linewidth=2, markersize=3,
                  color=colors[i], label="K=$k")
        end
    end
    
    return p
end

# Plot for both distances
for d in target_distances
    if haskey(results, d) && !isempty(results[d]["k_values"])
        # Select a few representative K values
        k_vals = results[d]["k_values"]
        k_to_plot = [k_vals[1], k_vals[div(length(k_vals), 2)], k_vals[end]]
        k_to_plot = unique(k_to_plot)  # Remove duplicates
        
        p = plot_fidelity_curves(data_dict, d, k_to_plot, threshold)
        display(p)
    end
end

## Save Results

In [ ]:
# Save the crossing analysis results
output_file = "crossing_analysis_results.jld2"
save(output_file, "results", results, "threshold", threshold, "target_distances", target_distances)
println("Results saved to $output_file")

# Also save as a simple text summary
open("crossing_analysis_summary.txt", "w") do io
    println(io, "Fidelity Crossing Analysis Summary")
    println(io, "Threshold: $threshold")
    println(io, "Target distances: $target_distances")
    println(io, "")
    
    for d in target_distances
        if haskey(results, d) && !isempty(results[d]["k_values"])
            println(io, "Distance d=$d:")
            k_vals = results[d]["k_values"]
            crossing_vals = results[d]["crossing_sigmas"]
            
            for (k, sigma) in zip(k_vals, crossing_vals)
                println(io, "  K=$k: σ_crossing = $sigma")
            end
            println(io, "")
        end
    end
end

println("Summary saved to crossing_analysis_summary.txt")